In [1]:
import pandas as pd

df = pd.read_csv('../2. Dataset/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
def tokenize(text):

  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  
  return text.split()

In [3]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
vocab = {'<UNK>':0}

In [5]:
def build_vocab(row):
  
  tokenized_question = tokenize(row['question'])
  
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      
      vocab[token] = len(vocab)

In [6]:
df.apply(build_vocab, axis=1) # for each row

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [7]:
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

In [9]:
class QADataset(Dataset):

  def __init__(self, df, vocab):

    self.df = df
    self.vocab = vocab

  def __len__(self):
    
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [10]:
dataset = QADataset(df, vocab)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [11]:
q , a = dataset[0]
print(q.shape , a.shape)

torch.Size([6]) torch.Size([1])


In [14]:
import torch.nn as nn

class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):

    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)  
    self.rnn = nn.RNN(50, 64, batch_first=True)
    #  nn.RNN(50, 64, batch_first=True, nonlinearity='relu') ,
    #  you can do this to change activation function , default is tanh
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):

    embedded_question = self.embedding(question)  # this is flexible like you can pass in batch , seq or even a singel word
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))  

    # ex : (2,3) => (2,3,50) => hidden : (2,3,64) , final : (1,2,64) => final then squezze : (2,vocab size)

    return output

In [15]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


write about c , d

In [18]:
learning_rate = 0.001
epochs = 20

In [19]:
model = SimpleRNN(len(vocab))

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    output = model(question)

    # loss -> output shape (1,324) - (1)
    # inside CrossEntropyLoss => Apply Softmax → convert logits → probabilities

    loss = criterion(output, answer[0])

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 528.090919
Epoch: 2, Loss: 458.714390
Epoch: 3, Loss: 378.145655
Epoch: 4, Loss: 319.229693
Epoch: 5, Loss: 268.699638
Epoch: 6, Loss: 219.591051
Epoch: 7, Loss: 175.431788
Epoch: 8, Loss: 136.682385
Epoch: 9, Loss: 105.355241
Epoch: 10, Loss: 79.890064
Epoch: 11, Loss: 61.585481
Epoch: 12, Loss: 48.281191
Epoch: 13, Loss: 38.078587
Epoch: 14, Loss: 31.051787
Epoch: 15, Loss: 25.467780
Epoch: 16, Loss: 21.294527
Epoch: 17, Loss: 17.820949
Epoch: 18, Loss: 15.242909
Epoch: 19, Loss: 13.066979
Epoch: 20, Loss: 11.353748


In [25]:
def predict(model, question, threshold=0.5):

  numerical_question = text_to_indices(question, vocab)

  print("numerical_question" ,len(numerical_question) )

  question_tensor = torch.tensor(numerical_question).unsqueeze(0) # adding  a batch dim

  print("question_tensor" ,question_tensor.shape )

  output = model(question_tensor)

  print("output" , output.shape)

  probs = torch.nn.functional.softmax(output, dim=1)  # along row

  print("probs" , probs.shape)

  value, index = torch.max(probs, dim=1)  

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [26]:
predict(model, "What is the largest planet in our solar system?")

numerical_question 9
question_tensor torch.Size([1, 9])
output torch.Size([1, 324])
probs torch.Size([1, 324])
jupiter
